In [1]:
import os, sys, email
import pandas as pd
import numpy as np
import re

import networkx as nx
import pyvis
from nltk.tokenize.regexp import RegexpTokenizer
from pyvis.network import Network

In [2]:
emails_df = pd.read_csv("emails.csv")
len(emails_df)

517401

In [3]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [4]:
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
#Getting the contents of email
def get_content_of_email(body):
    parts = []
    for part in body.walk():
        if part.get_content_type() == 'text/plain': 
            parts.append(part.get_payload())
    return ''.join(parts)

In [6]:
#Getting email address
def get_email_address(line): 
    if line:
        address = line.split(',')
        address = frozenset(map(lambda x : x.strip(), address))
    else:
        address = None
    return address

In [7]:
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True) 

fields = messages[0].keys()
for field in fields:
    emails_df[field] = [doc[field] for doc in messages]
    
emails_df['content'] = list(map(get_content_of_email, messages))

emails_df['From'] = emails_df['From'].map(get_email_address)
emails_df['To'] = emails_df['To'].map(get_email_address)

emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

In [8]:
def remove_forwarded_by(text):
    condition = "[- ]*Forwarded by[\S\s]*Subject:[\S\t ]*"
    return re.sub(condition, "", text).strip()

In [9]:
# Getting originial emails
emails_df["content"] = emails_df.content.map(remove_forwarded_by)
emails_df = emails_df[emails_df["content"].str.contains("-----Original Message-----")]

In [10]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69514 entries, 942 to 517400
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   file                       69514 non-null  object
 1   Message-ID                 69514 non-null  object
 2   Date                       69514 non-null  object
 3   From                       69514 non-null  object
 4   To                         69062 non-null  object
 5   Subject                    69514 non-null  object
 6   Mime-Version               69514 non-null  object
 7   Content-Type               69514 non-null  object
 8   Content-Transfer-Encoding  69514 non-null  object
 9   X-From                     69514 non-null  object
 10  X-To                       69514 non-null  object
 11  X-cc                       69514 non-null  object
 12  X-bcc                      69514 non-null  object
 13  X-Folder                   69514 non-null  object
 14  X-O

In [11]:
emails_df = emails_df.set_index('Message-ID').drop(['file','Mime-Version','Content-Type', 
                                                    'Content-Transfer-Encoding','Subject', 'X-FileName', 'X-Folder',
                                                    'X-Origin', 'X-FileName', 'X-cc', 'X-bcc' ], axis=1)
emails_df['Date'] = pd.to_datetime(emails_df['Date'], utc = True)
emails_df['Date'] = emails_df['Date'].dt.date

In [12]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69514 entries, <2194589.1075855666394.JavaMail.evans@thyme> to <28618979.1075842030037.JavaMail.evans@thyme>
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     69514 non-null  object
 1   From     69514 non-null  object
 2   To       69062 non-null  object
 3   X-From   69514 non-null  object
 4   X-To     69514 non-null  object
 5   content  69514 non-null  object
 6   user     69514 non-null  object
dtypes: object(7)
memory usage: 4.2+ MB


In [13]:
emails_df.user.nunique()

149

In [14]:
emails_df['From'].nunique()

5221

In [15]:
emails_df['To'].nunique()

15443

In [16]:
emails_df.head()

,Date,From,To,X-From,X-To,content,user
Message-ID,,,,,,,
<2194589.1075855666394.JavaMail.evans@thyme>,2000-12-11,(jsmith@austintx.com),(phillip.k.allen@enron.com),"""Jeff Smith"" <jsmith@austintx.com>",<Phillip.K.Allen@enron.com>,I WILL TALK TO LUTZ ABOUT HIS SHARE OF THE LEG...,allen-p
<8914065.1075858632242.JavaMail.evans@thyme>,2001-10-18,(no.address@enron.com),None,Enron Global Technology@ENRON,All Enron Worldwide@ENRON <??SAll Enron Worldw...,"Earlier this week, Enron Global Technology ann...",allen-p
<6254563.1075858632387.JavaMail.evans@thyme>,2001-10-19,(bodyshop@enron.com),(bodyshop@enron.com),Bodyshop </O=ENRON/OU=NA/CN=RECIPIENTS/CN=BODY...,Bodyshop </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Body...,"-----Original Message-----\nFrom: \tWright, Le...",allen-p
<3461377.1075858633882.JavaMail.evans@thyme>,2001-10-22,(w..cantrell@enron.com),"(matt.smith@enron.com, patti.sullivan@enron.co...","Cantrell, Rebecca W. </O=ENRON/OU=NA/CN=RECIPI...","Lawner, Leslie </O=ENRON/OU=NA/CN=RECIPIENTS/C...",Updated to include comments received as of Fri...,allen-p
<8996818.1075858633953.JavaMail.evans@thyme>,2001-10-22,(mike.grigsby@enron.com),"(matt.smith@enron.com, k..allen@enron.com, l.....","Grigsby, Mike </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","-----Original Message-----\nFrom: \tSheppard, ...",allen-p


In [17]:
emails_df = emails_df.reset_index(drop = True)

In [18]:
emails_df['Date'] = pd.to_datetime(emails_df['Date'], utc = True)

In [19]:
emails_df['year'] = emails_df['Date'].dt.year

In [20]:
emails_df['month'] = emails_df['Date'].dt.month

In [21]:
emails_df.head()

,Date,From,To,X-From,X-To,content,user,year,month
0,2000-12-11 00:00:00+00:00,(jsmith@austintx.com),(phillip.k.allen@enron.com),"""Jeff Smith"" <jsmith@austintx.com>",<Phillip.K.Allen@enron.com>,I WILL TALK TO LUTZ ABOUT HIS SHARE OF THE LEG...,allen-p,2000,12
1,2001-10-18 00:00:00+00:00,(no.address@enron.com),None,Enron Global Technology@ENRON,All Enron Worldwide@ENRON <??SAll Enron Worldw...,"Earlier this week, Enron Global Technology ann...",allen-p,2001,10
2,2001-10-19 00:00:00+00:00,(bodyshop@enron.com),(bodyshop@enron.com),Bodyshop </O=ENRON/OU=NA/CN=RECIPIENTS/CN=BODY...,Bodyshop </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Body...,"-----Original Message-----\nFrom: \tWright, Le...",allen-p,2001,10
3,2001-10-22 00:00:00+00:00,(w..cantrell@enron.com),"(matt.smith@enron.com, patti.sullivan@enron.co...","Cantrell, Rebecca W. </O=ENRON/OU=NA/CN=RECIPI...","Lawner, Leslie </O=ENRON/OU=NA/CN=RECIPIENTS/C...",Updated to include comments received as of Fri...,allen-p,2001,10
4,2001-10-22 00:00:00+00:00,(mike.grigsby@enron.com),"(matt.smith@enron.com, k..allen@enron.com, l.....","Grigsby, Mike </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","-----Original Message-----\nFrom: \tSheppard, ...",allen-p,2001,10


## who send most emails to whom

In [22]:
user = ['derrick-j', 'forney-j', 'lay-k', 'delainey-d', 'skilling-j']

In [23]:
emails_df = emails_df[emails_df['user'].isin(user)]

In [24]:
emails_df.head()

,Date,From,To,X-From,X-To,content,user,year,month
12740,2001-12-28 00:00:00+00:00,(shona.wilson@enron.com),"(dimitri.taylor@enron.com, wanda.curry@enron.c...","Wilson, Shona </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Abel, Chris </O=ENRON/OU=NA/CN=RECIPIENTS/CN=C...",The preliminary should be out by the end of th...,delainey-d,2001,12
12741,2001-12-28 00:00:00+00:00,(tharsilla.broussard@enron.com),(w..delainey@enron.com),"Broussard, Tharsilla </O=ENRON/OU=NA/CN=RECIPI...","Delainey, David W. </O=ENRON/OU=NA/CN=RECIPIEN...",You're very welcome! You have a good new year...,delainey-d,2001,12
12742,2001-12-27 00:00:00+00:00,(s..muller@enron.com),(w..delainey@enron.com),"Muller, Mark S. </O=ENRON/OU=NA/CN=RECIPIENTS/...","Delainey, David W. </O=ENRON/OU=NA/CN=RECIPIEN...",aren't you supposed to be skiing\n\n-----Origi...,delainey-d,2001,12
12743,2001-12-23 00:00:00+00:00,(no.address@enron.com),"(w..delainey@enron.com, jeffrey.mcmahon@enron....","Miller, Don (Asset Mktg) </O=ENRON/OU=NA/CN=RE...","Dietrich, Janet </O=ENRON/OU=NA/CN=RECIPIENTS/...",I will track down more details this week. \n ...,delainey-d,2001,12
12744,2002-01-10 00:00:00+00:00,(david.oxley@enron.com),"(terrie.wheeler@enron.com, kriste.sullivan@enr...","Oxley, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...","Dietrich, Janet </O=ENRON/OU=NA/CN=RECIPIENTS/...",OK. We actually had here on NETCo list. I'll r...,delainey-d,2002,1


In [25]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1717 entries, 12740 to 57258
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   Date     1717 non-null   datetime64[ns, UTC]
 1   From     1717 non-null   object             
 2   To       1670 non-null   object             
 3   X-From   1717 non-null   object             
 4   X-To     1717 non-null   object             
 5   content  1717 non-null   object             
 6   user     1717 non-null   object             
 7   year     1717 non-null   int64              
 8   month    1717 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 134.1+ KB


In [26]:
emails_df['From'].nunique()

335

In [27]:
emails_df['To'].nunique()

533

In [28]:
year = [2001]

In [29]:
emails_2001 = emails_df[emails_df['year'].isin(year)]

# Enron data [Jan-june'2001]

In [30]:
month = [1, 2, 3, 4, 5, 6]

In [31]:
emails_rise = emails_2001[emails_2001['month'].isin(month)]

In [32]:
sub_df_rise = emails_rise[['From', 'To', 'Date']].dropna()
print(sub_df_rise.shape)

sub_df_rise = sub_df_rise.loc[sub_df_rise['To'].map(len) == 1]
print(sub_df_rise.shape)

(506, 3)
(437, 3)


In [33]:
sub_df_rise = sub_df_rise.groupby(['From', 'To']).count().reset_index()

sub_df_rise['From'] = sub_df_rise['From'].map(lambda x: next(iter(x)))
sub_df_rise['To'] = sub_df_rise['To'].map(lambda x: next(iter(x)))

sub_df_rise.rename(columns={'Date': 'count'}, inplace=True)
sub_df_rise.sort_values('count', ascending=False).head(10)

,From,To,count
127,james.derrick@enron.com,j.harris@enron.com,93
35,rob.walls@enron.com,james.derrick@enron.com,22
130,james.derrick@enron.com,rob.walls@enron.com,13
44,sherri.sera@enron.com,eharris@insightpartners.com,12
31,billy.lemmons@enron.com,jeff.skilling@enron.com,10
32,eharris@insightpartners.com,jeff.skilling@enron.com,9
83,fparra@swbanktx.com,jeff.skilling@enron.com,7
117,james.derrick@enron.com,rex.rogers@enron.com,7
76,rosalee.fleming@enron.com,barbara.paige@enron.com,6
123,james.derrick@enron.com,sharon.butcher@enron.com,6


In [34]:
sub_df_rise.to_csv("sub_df_rise.csv")

In [35]:
G = nx.from_pandas_edgelist(sub_df_rise,
                             source = 'From',
                             target = 'To',
                             edge_attr = 'count')

In [36]:
net = Network('700px', '1300px', notebook=True)
net.from_nx(G)
net.show("network_rise.html")

# Enron data [July-Dec'2001]

In [37]:
month2 = [7, 8, 9, 10, 11]

In [38]:
emails_fall = emails_2001[emails_2001['month'].isin(month2)]

In [39]:
sub_df_fall = emails_fall[['From', 'To', 'Date']].dropna()
print(sub_df_fall.shape)

sub_df_fall = sub_df_fall.loc[sub_df_fall['To'].map(len) == 1]
print(sub_df_fall.shape)

(816, 3)
(635, 3)


In [40]:
sub_df_fall = sub_df_fall.groupby(['From', 'To']).count().reset_index()

sub_df_fall['From'] = sub_df_fall['From'].map(lambda x: next(iter(x)))
sub_df_fall['To'] = sub_df_fall['To'].map(lambda x: next(iter(x)))

sub_df_fall.rename(columns={'Date': 'count'}, inplace=True)
sub_df_fall.sort_values('count', ascending=False).head(10)

,From,To,count
120,james.derrick@enron.com,c..williams@enron.com,77
119,james.derrick@enron.com,j.harris@enron.com,50
165,j.harris@enron.com,james.derrick@enron.com,19
114,james.derrick@enron.com,rob.walls@enron.com,15
270,m..forney@enron.com,jeffrey.miller@enron.com,13
75,chuck.paul@wiseshops.com,smu-betas@yahoogroups.com,11
169,rob.walls@enron.com,james.derrick@enron.com,9
239,m..forney@enron.com,scott.jacobucci@elpaso.com,8
118,james.derrick@enron.com,rex.rogers@enron.com,8
273,m..forney@enron.com,lloyd.will@enron.com,7


In [41]:
sub_df_fall.to_csv("sub_df_fall.csv")

In [42]:
G2 = nx.from_pandas_edgelist(sub_df_fall,
                             source = 'From',
                             target = 'To',
                             edge_attr = 'count')

In [43]:
net = Network('700px', '1300px', notebook=True)
net.from_nx(G2)
net.show("network_fall.html")